In [1]:
# import
import pandas as pd
import numpy as np
from simple.plotly import interactFigure, updateLines, updateSliders, chartParallel
from numba import njit
from simple.funcs import vwap, vPIN
from simple.geneopt import GeneOpt
from ipyslickgrid import show_grid
from multiprocessing import current_process
import plotly.graph_objs as go
from ipywidgets import VBox

In [2]:
T = np.load('data/tick.npz')['BTCUSDT'].view(np.recarray)
len(T)

889360

In [3]:
fp32 = np.float32
default_fee = 0.05


@njit(nogil=True)
def backtestLimit(PriceA, qA, qB, signal, threshold, fee_percent=default_fee) -> list:
    """Vectorized backtester for limit order strategies"""

    buys = [(int(x), fp32(x)) for x in range(0)]
    sells = [(int(x), fp32(x)) for x in range(0)]
    trades = [(int(x), fp32(x), int(x), fp32(x), int(x), fp32(x), fp32(x)) for x in range(0)]

    pos: int = 0

    for i in range(len(PriceA) - 1):
        price = PriceA[i]

        if price > qA[i] and signal[i] > threshold:
            delta_pos = -min(pos + 1, 1)
        elif price < qB[i] and signal[i] < -threshold:
            delta_pos = min(1 - pos, 1)
        else:
            delta_pos = 0

        k = i + 1
        if delta_pos > 0:
            buys.append((k, qB[k]))
        elif delta_pos < 0:
            sells.append((k, qA[k]))

        if len(sells) > 0 and len(buys) > 0:
            k_buy, buy = buys.pop(0)
            k_sell, sell = sells.pop(0)
            d_rawPnL = sell - buy
            fee = fee_percent / 100 * (sell + buy)
            d_PnL = d_rawPnL - fee
            if delta_pos < 0:
                trades.append((k_buy, buy, k_sell, sell, -delta_pos, d_PnL, fee))
            else:
                trades.append((k_sell, sell, k_buy, buy, -delta_pos, d_PnL, fee))

        pos += delta_pos

    return trades


def npBacktestLimit(PriceA, qA, qB, signal, threshold, fee_percent=default_fee) -> np.ndarray:
    """Converts trades from the limit-backtester to structured array"""

    trades = backtestLimit(PriceA, qA, qB, signal, threshold, fee_percent=fee_percent)
    TPairTrade = [('X0', int), ('Price0', float), ('X1', int), ('Price1', float),
                  ('Size', float), ('Profit', float), ('Fee', float)]
    return np.array(trades, dtype=TPairTrade).view(np.recarray)


def getLong(trades):
    LongEntry = trades[['X0', 'Price0']][trades.Size > 0]
    LongExit = trades[['X1', 'Price1']][trades.Size < 0]
    return {'x': np.concatenate((LongEntry.X0, LongExit.X1)), 
            'y': np.concatenate((LongEntry.Price0, LongExit.Price1))}


def getShort(trades):
    ShortEntry = trades[['X0', 'Price0']][trades.Size < 0]
    ShortExit = trades[['X1', 'Price1']][trades.Size > 0]
    return {'x': np.concatenate((ShortEntry.X0, ShortExit.X1)),
            'y': np.concatenate((ShortEntry.Price0, ShortExit.Price1))}

In [4]:
# declare chart linestyles
line_styles = {
    'Tick': dict(color='gray', opacity=0.25),
    'Center': dict(color='blue', opacity=0.5),
    'qA': dict(color='red', opacity=0.5, dash='dot'),
    'qB': dict(color='green', opacity=0.5, dash='dot'),
    'Profit': dict(color='black', width=8, opacity=0.1, secondary_y=True, shape='hv'),
    'Buy': dict(mode='markers', color='green', symbol='triangle-up', size=10, line=dict(color="darkgreen", width=1)),
    'Sell': dict(mode='markers', color='red', symbol='triangle-down', size=10, line=dict(color="darkred", width=1)),
    'OSC': dict(color='orange', row=2, col=1, opacity=0.6),
}

In [5]:
def model(Period: int = (1000, 50000), StdDev: float = (1, 4, 0.1), Threshold: int = (0, 40)):
    if Period == 0 or StdDev == 0: return 0, {}
    Tick = T.PriceA
    Center = vwap(Tick, T.VolumeA, Period)
    std = pd.Series(Tick).rolling(Period).std().bfill().values
    qA = Center + std*StdDev
    qB = Center - std*StdDev
    OSC = vPIN(T, Period)

    trades = npBacktestLimit(Tick, qA, qB, OSC, Threshold)
    Buy, Sell = getLong(trades), getShort(trades)
    Profit = {'x': trades.X1, 'y': trades.Profit.cumsum()}
    
    return trades.Profit.sum(), {
        'Count': len(trades),
        'AvgProfit': trades.Profit.mean() if len(trades) > 0 else 0,
        'Sharpe': trades.Profit.sum() / trades.Profit.std() if len(trades) > 1 else 0
     } if current_process().daemon else locals()

In [6]:
# Genetic optimizer
G = GeneOpt(model)
G.maximize(population_size=256, generations=5)

  0%|          | 0/5 [00:00<?, ?it/s]

{'Period': 26579, 'StdDev': 2.3540066349175297, 'Threshold': 2}

In [7]:
# Create interactive figure
box = interactFigure(model, line_styles, rows=2, height=650)

In [8]:
# Genetic result browser
X = pd.DataFrame(G.log, columns=G.log_columns).drop_duplicates().sort_values('Fitness')


def on_changed(event, grid):
    changed = grid.get_changed_df()
    k = event['new'][0]
    selected = changed.iloc[k:k+1].to_dict('records')[0]
    param = dict(filter(lambda x: x[0] in G.args, selected.items()))

    updateSliders(box.children[0].children, **param)
    updateLines(box.children[1], **model(**param)[1])


grid = show_grid(X, grid_options={'editable': False, 'forceFitColumns': True, 'multiSelect': False}, 
                 column_options={'defaultSortAsc': False})
grid.on('selection_changed', on_changed)

VBox([box, grid])

In [9]:
chartParallel(X)

FigureWidget({
    'data': [{'dimensions': [{'label': 'Period',
                              'values': array(…